In [4]:
import pandas as pd
import numpy as np

In [14]:
# PC
data = pd.read_csv(r'C:\Users\Ezequiel\Desktop\machine-learning-2c-2025\data.csv', low_memory=False, sep='|')
# Notebook
#data = pd.read_csv(r'C:\Users\forma\Desktop\machine-learning-2c-2025\data.csv', low_memory=False, sep='|')

In [15]:
data.columns

Index(['client_id', 'Target', 'Month', 'First_product_dt', 'Last_product_dt',
       'CreditCard_Premium', 'CreditCard_Active', 'CreditCard_CoBranding',
       'Loan_Active', 'Mortgage_Active', 'SavingAccount_Active_ARG_Salary',
       'SavingAccount_Active_ARG', 'SavingAccount_Active_DOLLAR',
       'DebitCard_Active', 'Investment_Active', 'Package_Active',
       'Insurance_Life', 'Insurance_Home', 'Insurance_Accidents',
       'Insurance_Mobile', 'Insurance_ATM', 'Insurance_Unemployment', 'Sex',
       'Client_Age_grp', 'SavingAccount_Balance_FirstDate',
       'SavingAccount_Balance_LastDate', 'SavingAccount_Balance_Average',
       'SavingAccount_Days_with_use', 'SavingAccount_Days_with_Credits',
       'SavingAccount_Days_with_Debits',
       'SavingAccount_Salary_Payment_Transactions',
       'SavingAccount_Transfer_In_Transactions',
       'SavingAccount_ATM_Extraction_Transactions',
       'SavingAccount_Service_Payment_Transactions',
       'SavingAccount_CreditCard_Payment_T

In [16]:
print(data['client_id'].value_counts())

client_id
4221845                   9
7219504                   9
5882370                   9
1158013                   9
3914580                   9
                         ..
6623284                   1
672542                    1
264018                    1
4424661                   1
(238615 rows affected)    1
Name: count, Length: 26561, dtype: int64


In [27]:
# Busco los ID´s de los clientes que tienen 9 meses de data

# Creo una tabla con ID_cliente, cantidad de apariciones
ids_clientes_9m = data['client_id'].value_counts()

# Me quedo solo con aquellos que aparecen 9 veces 
ids_clientes_9m = ids_clientes_9m[ids_clientes_9m == 9].index

# Me quedo con todos los datos de quienes aparecen 9 meses
data_clientes_9m = data[data['client_id'].isin(ids_clientes_9m)]

data_clientes_9m.shape

(238347, 77)

In [22]:
# [agosto, septiembre, octubre, noviembre, diciembre, enero] TW, [febrero] LW, [marzo, abril] PW

# me quedo con los clientes que en el ultimo mes de la TW tienen package_active = no & CreditCard_CoBranding = no, y ademas, aparecen en los 9 meses
filtered_bussiness_conditions = data_clientes_9m[(data_clientes_9m['Month'] == '2019-01-01') 
                    & (data_clientes_9m['Package_Active'] == 'No')
                    & (data_clientes_9m['CreditCard_CoBranding'] == 'No')]

# Obtengo sus id
ids_filtered_bussiness_conditions = filtered_bussiness_conditions['client_id'].values
ids_filtered_bussiness_conditions.shape

(23191,)

In [23]:
# Ahora unifico la tabla de clientes filtrados por paquete con los que aparecen 9 meses 
data_final = data[data['client_id'].isin(ids_filtered_bussiness_conditions)]

data_final.shape

(208719, 77)

In [24]:
# Ahora ya puedo separar las ventanas
training_window = data_final[data_final['Month'].isin(['2018-08-01',    
'2018-09-01',
'2018-10-01',
'2018-11-01',
'2018-12-01',
'2019-01-01'   
])]

training_window.shape

(139146, 77)

In [25]:
#data_lead_window = data_final[data_final['Month'].isin(['2018-08-01'])]
#data_training_window.shape

In [26]:
# Ahora tengo que empezar a limpiar los datos, empiezo por ver las columnas que contengan nulos
training_window.columns[training_window.isnull().any()].tolist()

['SavingAccount_Balance_Average', 'Region', 'CreditCard_Product']

In [29]:
# 3 columnas tienen nulos

# 1) SavingAccount_Balance_Average: Esta es una de las columnas que presenta nulos, es el promedio del monto que se tiene en la cuenta de ahorros
print(training_window['SavingAccount_Balance_Average'])

0          406.35
1           73.56
2            0.00
3         2366.02
4            0.40
           ...   
238597       0.00
238603       0.00
238604     367.43
238608       0.00
238612        NaN
Name: SavingAccount_Balance_Average, Length: 139146, dtype: float64


In [31]:
# Reemplazo los valores nulos con Balance_FirstDate - Debits + Credits
training_window.loc[:, 'SavingAccount_Balance_Average'] = (
    training_window['SavingAccount_Balance_Average'].fillna(
        training_window['SavingAccount_Balance_FirstDate']
        - training_window['SavingAccount_Debits_Amounts']
        + training_window['SavingAccount_Credits_Amounts']
    )
)

In [32]:
# 2) Region: Es la segunda columna con nulos 
print(training_window['Region'])

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
238597    NaN
238603    NaN
238604    NaN
238608    NaN
238612    NaN
Name: Region, Length: 139146, dtype: object


In [33]:
# Parecen todos nulos, me fijo si tiene valores diferentes
training_window['Region'].value_counts()

Series([], Name: count, dtype: int64)

In [36]:
data['Region'].value_counts()

Region
BUENOS AIRES                     14293
REGION CENTRO                    11735
REGION NORTE GRANDE ARGENTINO    10142
REGION PATAGONICA                 4968
CABA Centro/Norte                 4228
REGION CUYO                       3982
AMBA Resto                        3732
Name: count, dtype: int64

In [ ]:
# Preguntar, porque no se hace nada con region?

In [39]:
print(training_window['CreditCard_Product'])

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
238597    NaN
238603    NaN
238604    NaN
238608    NaN
238612    NaN
Name: CreditCard_Product, Length: 139146, dtype: object


In [37]:
# 3) CreditCard_Product: Es la terecer columna con nulos
training_window['CreditCard_Product'].value_counts()

Series([], Name: count, dtype: int64)

In [40]:
data['CreditCard_Product'].value_counts()

CreditCard_Product
J55660104XX012    17298
J55660202XX012    12047
J55660124XX012     4978
J55660102XX012      834
J55660123XX012      774
J55660702XX012      567
J55661002XX012      138
Name: count, dtype: int64

In [ ]:
# Preguntar, porque no se hace nada con CreditCard_Product?

In [ ]:
# Preguntar que hace en la linea 35?